In [109]:
import pandas as pd
from datetime import date, datetime, timedelta
import pandas
pandas.set_option('display.max_columns', None)

In [110]:
csv_files = [ '39data.csv' , 'ENG_Premier_League_elo.csv' , 
              '61data.csv' , 'FRA_Ligue_1_elo.csv', 
              '78data.csv' , 'GER_Bundesliga_elo.csv', 
              '135data.csv', 'ITA_Serie_A_elo.csv', 
              '140data.csv', 'ESP_La_Liga_elo.csv']

In [120]:
def addWeek(data):  
    # Define the variable to be incremented
    starting = date.fromisoformat('2010-08-07')
    data['fixture_date'] = pd.to_datetime(data['fixture_date'])

    # check in which date interval the current date belongs
    def assign_week(row):
        nonlocal starting
        if (row['fixture_date'].date() - starting).days < 7:
            return starting
        else:
            while (row['fixture_date'].date() - starting).days > 7:
                if starting.month == 7:
                    starting = starting.replace(month=8, day=7)
                else:
                    starting += timedelta(days=7)
            starting += timedelta(days=7)
            return starting

    # add the new week column
    data['week'] = data.apply(assign_week, axis=1)

In [124]:
def joinOnWeek(fixture, elo, fixt_team):
  fixture['week'] = pd.to_datetime(fixture['week'])
  elo['week'] = pd.to_datetime(elo['week'])
  merged = pd.merge(fixture, elo,  left_on=[fixt_team, 'week'], right_on=['team', 'week'])
  return merged

In [51]:
def dropColumns(fixture, columns_to_drop):
  #columns_to_drop = ['team', 'country', 'league']
  fixture = fixture.drop(columns=columns_to_drop, axis=1)
  return fixture

In [52]:
def RenameElo(fixture, old_elo, new_elo, old_rank, new_rank):
  fixture.rename(columns={old_elo: new_elo}, inplace=True)
  fixture.rename(columns={old_rank: new_rank}, inplace=True)
  return fixture

In [53]:
def ReturnJoinedData(fixture, elo):
  addWeek(fixture)
  home = joinOnWeek(fixture, elo, 'teams_home_name')
  home = dropColumns(home, columns_to_drop = ['team', 'country', 'league'])
  home = RenameElo(home, 'elo', 'home_elo', 'rank', 'home_rank')

  away = joinOnWeek(home, elo, 'teams_away_name')
  away = dropColumns(away, columns_to_drop = ['team', 'country', 'league', 'week'])
  away = RenameElo(away, 'elo', 'away_elo', 'rank', 'away_rank')
  return away

In [54]:
for i in range(0, len(csv_files), 2):
  data = pd.read_csv(csv_files[i])
  elo = pd.read_csv(csv_files[i + 1])

  FixElo = ReturnJoinedData(data, elo)
  FixElo.to_csv('C:\\Users\\Imad\\Desktop\\S3\\BigData\\YaFaa-Flask\\YAFAA_Fixtures_Elo\\Fixtures_' + csv_files[i + 1], index=False)  